In [2]:
import sys
sys.path.append("../../..")
sys.path.append("../..")
from module import load_token_mapping
from tqdm import tqdm
import dgl
import torch

In [2]:
def isOneEditDistance(s: str, t: str) -> bool:
    distance = len(s) - len(t)
    if abs(distance) > 1:
        return False
    if not s or not t:
        return s != t
    
    edit = 0
    i,j = 0,0
    while i < len(s) and j < len(t):
        if s[i] == t[j]:
            i += 1
            j += 1
        else:
            if edit:
                return False

            if distance == 1: # 删除
                i += 1
            elif distance == -1:  # 插入
                j += 1
            else:   # 替换
                i += 1
                j += 1 
            edit += 1
    if i < len(s):
        return edit == 0
    if j < len(t):
        return edit == 0
    return i == len(s) and j == len(t) and edit == 1

In [5]:
i2q, q2i  = load_token_mapping('queries')

In [7]:
with open('one_edit.graph', 'w') as f:
    # for i in tqdm(range(len(i2q))):
    for i in tqdm(range(10)):
        for j in range(i + 1, len(i2q)):
            if isOneEditDistance(i2q[i], i2q[j]):
                f.write(f'{i2q[i]}\t{i2q[j]}\n')

100%|██████████| 10/10 [00:01<00:00,  9.11it/s]


In [8]:
#####build query-spu graph############
i2q, q2i  = load_token_mapping('queries')
i2s, s2i = load_token_mapping('spus')
s2i = {k: v + len(i2q) for k, v in s2i.items()}
src, dst = [], []
with open('new_train', 'r') as f:
    for line in f:
        query, spu, label = line.strip().split('\t')
        query = query.strip()
        spu = spu.strip()
        label = int(label)
        if label == 1:
            src.append(q2i[query])
            dst.append(s2i[spu])
g = dgl.graph((src, dst), num_nodes=len(i2q)+len(i2s))
g = dgl.to_bidirected(g)
g = dgl.add_self_loop(g)
dgl.save_graphs('q_s_train.graph', [g])


In [3]:
g, _ = dgl.load_graphs('q_s_train.graph')
g = g[0]

In [3]:
g.num_nodes()

1896833

In [8]:
g.edges()

(tensor([      1,       2,       3,  ..., 1896830, 1896831, 1896832]),
 tensor([1450546,  364874, 1612367,  ..., 1896830, 1896831, 1896832]))

In [9]:
g.edge_ids([1, 2], [1450546,  364874])

tensor([0, 1])

In [5]:
sampler = dgl.dataloading.as_edge_prediction_sampler(dgl.dataloading.NeighborSampler(fanouts=[2]))

In [6]:
_, _, blocks = sampler.sample(g, [0])

In [8]:
blocks[-1].dstdata[dgl.NID]

tensor([      1, 1450546])

In [20]:
g.edges(form='all')

(tensor([0, 0, 1, 2]), tensor([0, 1, 2, 0]), tensor([0, 1, 2, 3]))

In [18]:
bg1.edges(form='all')

(tensor([0, 0, 1, 2, 1, 2, 0]),
 tensor([0, 1, 2, 0, 0, 1, 2]),
 tensor([0, 1, 2, 3, 4, 5, 6]))

# 构建s2s图

In [19]:
from dgl import save_graphs, load_graphs
import dgl
import torch

In [2]:
pos_neg_graph, _ = load_graphs('pos_neg_train.graph')
pos_neg_graph = pos_neg_graph[0]

In [3]:
pos_g = pos_neg_graph['pos']

In [13]:
pos_g

Graph(num_nodes={'query': 218758, 'spu': 1678075},
      num_edges={('query', 'pos', 'spu'): 1151352},
      metagraph=[('query', 'spu', 'pos')])

In [14]:
min(pos_g.edges()[1])

tensor(0)

In [6]:
g = dgl.heterograph({
   ('query', 'q2s', 'spu'): (pos_g.edges()[0], pos_g.edges()[1]),
   ('spu', 's2q', 'query'): (pos_g.edges()[1], pos_g.edges()[0])
})

In [12]:
new_g = dgl.metapath_reachable_graph(g, ['s2q', 'q2s'])

In [15]:
new_g

Graph(num_nodes=1678075, num_edges=779348868,
      ndata_schemes={}
      edata_schemes={})

In [16]:
new_g = dgl.remove_self_loop(new_g)
new_g = dgl.add_self_loop(new_g)
new_g = dgl.to_homogeneous(new_g)

In [17]:
save_graphs('train_s2s.graph', [new_g], None)

In [21]:
torch.mean(new_g.in_degrees().float())

tensor(464.8886)